In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import h5py
import copy

import amici
from petab.C import *
import petab
import petab.C
import pypesto
import pypesto.petab
from pypesto.optimize import minimize
from pypesto.startpoint import uniform
from pypesto.engine import MultiProcessEngine, MultiThreadEngine
from pypesto.optimize.optimizer import FidesOptimizer
import pypesto.optimize as optimize

from pypesto.visualize import waterfall
from pypesto.visualize import parameters
from pypesto.visualize.model_fit import visualize_optimized_model_fit
from pypesto.visualize import profiles

import pypesto.profile as profile
from pypesto.optimize import ScipyOptimizer
from pypesto.profile.options import ProfileOptions

from pypesto.visualize.model_fit import visualize_optimized_model_fit, _get_simulation_rdatas

In [2]:
def hex_to_rgba_gradient(color1, color2, n):
    '''
    Create a gradient in rgba between two hex colors
    '''
    # Convert to rgba
    c1 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color1))
    c2 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color2))

    return [[(c1[i]*(n-j-1) + c2[i]*j)/(n-1) for i in range(4)] for j in range(n)]

# find the index for cut off based on Chi square distribution CI 95%
def find_cut_off_index(result, ci = 0.95):
    '''
    Find the cut off index for the data based on the Chi square distribution
    '''

    # calculate the chi square distribution
    cut_off_value = scipy.stats.chi2.ppf(ci, 1)

    # find the index
    best_fval = result.optimize_result.list[0].fval

    for i in range(len(result.optimize_result.list)):
        if result.optimize_result.list[i].fval > best_fval + cut_off_value:
            break
    
    return i - 1

def find_cut_off_x_trace(result, ci = 0.95, flatten = True):

    cut_off_value = scipy.stats.chi2.ppf(ci, 1)
    best_fval = result.optimize_result.list[0].fval

    # store the optimized x trace that are below the cut off value
    x_trace_within_cut_off = []
    if flatten:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off.append(x_trace[j])
    else:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            x_trace_within_cut_off_i = []
            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off_i.append(x_trace[j])
            x_trace_within_cut_off.append(x_trace_within_cut_off_i)

    return x_trace_within_cut_off

In [3]:
# Plot setting
plt.rcParams['font.size'] = 30

dpi = 100
wid = int(2560/dpi)
hei = int(1600/dpi)

In [4]:
# number of optimization runs
n_runs = 5000

In [5]:
np.random.seed(500)

In [6]:
# Define the folder where you want to save the figures
folder_path = "/Users/yuhongliu/Documents/OV/figures/ifac_hidden_model_corrected/n"+str(n_runs)+"_n=10/"

# If the folder does not exist, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# optimization
hierarchical = False

petab_yaml = 'petab_files/dividing_infected_cells_n=10.yaml'
petab.validate(petab_yaml)
petab_problem = petab.Problem.from_yaml(petab_yaml)

problem = pypesto.petab.PetabImporter(
        petab_problem,
        hierarchical=hierarchical,
        model_name=f"DIVIDING_INFECTED_CELLS_Model",
    ).create_problem(force_compile=True)

problem.objective.amici_model.setAllStatesNonNegative()

# some model properties
print("Model parameters:", list(problem.objective.amici_model.getParameterIds()), "\n")
print("Model const parameters:", list(problem.objective.amici_model.getFixedParameterIds()), "\n")
print("Model outputs:   ", list(problem.objective.amici_model.getObservableIds()), "\n")
print("Model states:    ", list(problem.objective.amici_model.getStateIds()), "\n")

Compiling amici model to folder /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model.
2024-11-11 11:37:31.159 - amici.petab.sbml_import - INFO - Importing model ...
2024-11-11 11:37:31.160 - amici.petab.sbml_import - INFO - Validating PEtab problem ...
2024-11-11 11:37:31.173 - amici.petab.sbml_import - INFO - Model name is 'DIVIDING_INFECTED_CELLS_Model'.
Writing model code to '/Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model'.
2024-11-11 11:37:31.174 - amici.petab.sbml_import - INFO - Species: 0
2024-11-11 11:37:31.174 - amici.petab.sbml_import - INFO - Global parameters: 22
2024-11-11 11:37:31.174 - amici.petab.sbml_import - INFO - Reactions: 0
2024-11-11 11:37:31.184 - amici.petab.sbml_import - INFO - Observables: 1
2024-11-11 11:37:31.184 - amici.petab.sbml_import - INFO - Sigmas: 1
2024-11-11 11:37:31.185 - amici.petab.sbml_import - DEBUG - Adding output

running build_ext
running AmiciBuildCMakeExtension
------------------------------ model_ext ------------------------------

==> Configuring:
$ cmake -S /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model -B /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model/build_model_ext -G Ninja -DCMAKE_MAKE_PROGRAM=/Users/yuhongliu/Documents/venv_ov/bin/ninja -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX:PATH=/Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model/DIVIDING_INFECTED_CELLS_Model -DCMAKE_VERBOSE_MAKEFILE=ON -DCMAKE_PREFIX_PATH=/Users/yuhongliu/Documents/venv_ov/lib/python3.12/site-packages/amici -DAMICI_PYTHON_BUILD_EXT_ONLY=ON -DPython3_EXECUTABLE=/Users/yuhongliu/Documents/venv_ov/bin/python3

==> Building:
$ cmake --build /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_mod

2024-11-11 11:37:44.617 - amici.petab.sbml_import - INFO - Finished Importing PEtab model            (1.35E+01s)


Model parameters: ['rho', 'kappa', 'psi', 'phi', 'alpha', 'beta', 'delta', 'sigma_add', 'sigma_mul'] 

Model const parameters: ['virus_injection'] 

Model outputs:    ['tumor_num'] 

Model states:     ['C_u', 'C_i', 'C_i1', 'C_i2', 'C_i3', 'C_i4', 'C_i5', 'C_i6', 'C_i7', 'C_i8', 'C_i9', 'C_i10', 'C_l', 'V'] 



# Gradient Check

In [9]:
startpoints = problem.get_startpoints(n_starts=2)

In [10]:
problem.objective.check_grad(
    x = startpoints[0],
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,-4.441204e+03,-4415.084051,-4467.315151,-4441.199601,52.231100,0.004861,1.094443e-06
kappa,3.767917e+02,402.392767,349.559677,375.976222,52.833090,0.815502,2.169025e-03
psi,-6.221777e+01,-33.494971,-90.829587,-62.162279,57.334616,0.055491,8.926841e-04
phi,-2.889060e+02,-260.399715,-317.554506,-288.977110,57.154791,0.071074,2.459497e-04
beta,-2.797584e-10,28.583408,-28.583402,0.000003,57.166811,0.000003,2.281700e-01
alpha,-5.439484e-03,28.562752,-28.584855,-0.011052,57.147607,0.005612,5.082727e-01
delta,6.030195e+00,34.859126,-22.584420,6.137353,57.443546,0.107158,1.745986e-02
sigma_add,-1.028216e+04,-10253.395099,-10310.918706,-10282.156902,57.523607,0.001284,1.248795e-07
sigma_mul,-1.967307e+03,-1938.739608,-1995.874339,-1967.306973,57.134731,0.000047,2.367705e-08


In [11]:
problem.objective.check_grad(
    x = problem.get_reduced_vector(startpoints[0]),
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,-4.441204e+03,-4415.084051,-4467.315151,-4441.199601,52.231100,0.004861,1.094443e-06
kappa,3.767917e+02,402.392767,349.559677,375.976222,52.833090,0.815502,2.169025e-03
psi,-6.221777e+01,-33.494971,-90.829587,-62.162279,57.334616,0.055491,8.926841e-04
phi,-2.889060e+02,-260.399715,-317.554506,-288.977110,57.154791,0.071074,2.459497e-04
beta,-2.797584e-10,28.583408,-28.583402,0.000003,57.166811,0.000003,2.281700e-01
alpha,-5.439484e-03,28.562752,-28.584855,-0.011052,57.147607,0.005612,5.082727e-01
delta,6.030195e+00,34.859126,-22.584420,6.137353,57.443546,0.107158,1.745986e-02
sigma_add,-1.028216e+04,-10253.395099,-10310.918706,-10282.156902,57.523607,0.001284,1.248795e-07
sigma_mul,-1.967307e+03,-1938.739608,-1995.874339,-1967.306973,57.134731,0.000047,2.367705e-08


In [12]:
gc = problem.objective.check_grad_multi_eps(
    x=problem.get_reduced_vector(startpoints[0]),
    verbosity=0,
    label='rel_err',  # default
)
gc

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,-4.441204e+03,-4415.084051,-4467.315151,-4.441200e+03,52.231100,4.860639e-03,1.094443e-06,1.000000e-05
kappa,3.767917e+02,386.428802,367.114078,3.767714e+02,19.314725,2.028410e-02,5.383649e-05,1.000000e-03
psi,-6.221777e+01,-61.970604,-62.464697,-6.221765e+01,0.494093,1.197811e-04,1.925226e-06,1.000000e-03
phi,-2.889060e+02,-289.637131,-288.177705,-2.889074e+02,1.459426,1.381873e-03,4.783115e-06,1.000000e-03
beta,-2.797584e-10,0.002858,-0.002858,-3.183231e-11,0.005717,2.479261e-10,2.479261e-09,1.000000e-01
alpha,-5.439484e-03,-0.002019,-0.008940,-5.479485e-03,0.006920,4.000127e-05,4.232020e-04,1.000000e-01
delta,6.030195e+00,6.323292,5.738966,6.031129e+00,0.584326,9.335293e-04,1.547595e-04,1.000000e-03
sigma_add,-1.028216e+04,-7423.814818,-13140.498954,-1.028216e+04,5716.684136,1.267750e-03,1.232961e-07,1.000000e-07
sigma_mul,-1.967307e+03,-1938.739608,-1995.874339,-1.967307e+03,57.134731,4.658002e-05,2.367705e-08,1.000000e-05


In [13]:
def highlight_value_above_threshold(x, threshold=10):
    return ['color: darkorange' if xi > threshold else None for xi in x]

gc.style.apply(
    highlight_value_above_threshold, subset=["fd_err"],
).background_gradient(
    cmap=sns.light_palette("purple", as_cmap=True), subset=["abs_err"],
).background_gradient(
    cmap=sns.light_palette("red", as_cmap=True), subset=["rel_err"],
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,-4441.204461,-4415.084051,-4467.315151,-4441.199601,52.231100,0.004861,0.000001,0.000010
kappa,376.791724,386.428802,367.114078,376.771440,19.314725,0.020284,0.000054,0.001000
psi,-62.217770,-61.970604,-62.464697,-62.217650,0.494093,0.000120,0.000002,0.001000
phi,-288.906036,-289.637131,-288.177705,-288.907418,1.459426,0.001382,0.000005,0.001000
beta,-0.000000,0.002858,-0.002858,-0.000000,0.005717,0.000000,0.000000,0.100000
alpha,-0.005439,-0.002019,-0.008940,-0.005479,0.006920,0.000040,0.000423,0.100000
delta,6.030195,6.323292,5.738966,6.031129,0.584326,0.000934,0.000155,0.001000
sigma_add,-10282.155618,-7423.814818,-13140.498954,-10282.156886,5716.684136,0.001268,0.000000,0.000000
sigma_mul,-1967.306927,-1938.739608,-1995.874339,-1967.306973,57.134731,0.000047,0.000000,0.000010
